In [19]:
import numpy as np
from scipy.stats import wishart

In [20]:
files_root = '/home/ruyi/cosmology/kcap_methods/tests/test_files/bandpower_cls/'

In [21]:
bin_order = ['bin_1_1', 'bin_2_1', 'bin_3_1', 'bin_4_1', 'bin_5_1',
             'bin_2_2', 'bin_3_2', 'bin_4_2', 'bin_5_2',
             'bin_3_3', 'bin_4_3', 'bin_5_3',
             'bin_4_4', 'bin_5_4',
             'bin_5_5']

ell = np.genfromtxt(files_root + 'ell.txt')

In [22]:
l_mode = 0
num_tom = 5
cls = np.zeros((num_tom, num_tom))

for i in range(num_tom):
    for j in range(num_tom):
        if j > i:
            cl = np.genfromtxt(files_root + 'bin_%s_%s.txt' %(j+1, i+1))[l_mode]
        else:
            cl = np.genfromtxt(files_root + 'bin_%s_%s.txt' %(i+1, j+1))[l_mode]
        cls[i, j] = cl

In [23]:
norm = 2*ell[l_mode] + 1
cls_resampled = wishart.rvs(df = norm, scale = cls / norm)

In [24]:
cl_binned = {}

for i in range(num_tom):
    for j in range(num_tom):
        if j <= i:
            cl_binned['bin_%s_%s' %(i+1, j+1)] = cls_resampled[i, j]
        else:
            pass

To calculate the covariance, follow the instructions on the joachimi and bridlle paper page 9, just set delta l = 1, and the shape noise I can get from the kids methodology paper. Same with the galaxy number density, but have to convert from arcmins to steradians. Then the survey area is also on the kids methodology paper. Use those values, plug it all in and then have a covariance per l. To get the bandpower covariance just sum it up per l and propagate the errors.

Then for the final big combined covariance you would just have a non zero diagonal with the squares of each covariance, but lots and lots of 0 off diagonal terms as the vector from 1 bin pair doesn't affect the signal of another bin pair, so it looks something like

A 0 0 0 0 0 \
0 B 0 0 0 0 \
0 0 C 0 0 0 \
0 0 0 D 0 0 \
0 0 0 0 E 0 \
0 0 0 0 0 F

In [25]:
ellip_disp = np.array([0.27, 0.26, 0.27, 0.25, 0.27])
n_eff = np.array([0.62, 1.18, 1.85, 1.26, 1.31]) * (60 **2) * (180**2) / (np.pi**2)#in steradian converted from arcmin

In [26]:
def get_cls(l_mode_index = 0, num_tom = 5, files_root = files_root):
    ell = np.genfromtxt(files_root + 'ell.txt')
    cls = np.zeros((num_tom, num_tom))

    for i in range(num_tom):
        for j in range(num_tom):
            if j > i:
                cl = np.genfromtxt(files_root + 'bin_%s_%s.txt' %(j+1, i+1))[l_mode_index]
            else:
                cl = np.genfromtxt(files_root + 'bin_%s_%s.txt' %(i+1, j+1))[l_mode_index]
            cls[i, j] = cl
    
    return ell, cls

def resample_cls(ell, cls, l_mode_index):
    norm = 2*ell[l_mode_index] + 1
    cls_resampled = wishart.rvs(df = norm, scale = cls / norm)
    
    return cls_resampled

def bin_cls(cls, num_tom):
    cl_binned = {}

    for i in range(num_tom):
        for j in range(num_tom):
            if j <= i:
                cl_binned['bin_%s_%s' %(i+1, j+1)] = cls[i, j]
            else:
                pass
    
    return cl_binned

def add_noise(cls, bin_1, bin_2, 
              ellip_disp = np.array([0.27, 0.26, 0.27, 0.25, 0.27]), 
              gal_num = np.array([0.62, 1.18, 1.85, 1.26, 1.31]) * (60 **2) * (180**2) / (np.pi**2)):
    try:
        cl = cls['bin_{}_{}'.format(bin_1,bin_2)]
    except:
        cl = cls['bin_{}_{}'.format(bin_2,bin_1)]
    
    if bin_1 == bin_2:
        cl += (ellip_disp[int(bin_1)-1]**2) / (2*gal_num[int(bin_1)-1])

    return cl
        
def cls_calc_covariance(cls, ell, l_mode_index, A):
    pre_factor = 2 * np.pi / (A * ell[l_mode_index])
    bin_keys = list(cls.keys())
    
    cl_ref = [["null" for _ in range(len(bin_keys))] for _ in range(len(bin_keys))]
    for i, bin_1 in enumerate(bin_keys):
        for j, bin_2 in enumerate(bin_keys):
            cl_ref[i][j] = bin_1.split("_")[-2] + bin_1.split("_")[-1] + bin_2.split("_")[-2] + bin_2.split("_")[-1]
            
    cl_cov = np.zeros((len(cl_binned), len(cl_binned)))

    for row in range(len(cl_binned)):
        for col in range(len(cl_binned)):
            i = cl_ref[row][col][0]
            j = cl_ref[row][col][1]
            k = cl_ref[row][col][2]
            l = cl_ref[row][col][3]
            
            cl_cov[row][col] = add_noise(cl_binned, i, k) * add_noise(cl_binned, j, l) + add_noise(cl_binned, i, l) * add_noise(cl_binned, j, k)
    
    cl_cov *= pre_factor
    
    return cl_cov

In [27]:
ell, theory_cls = get_cls(l_mode_index = 300, num_tom = 5, files_root = files_root)
one_draw_cls = resample_cls(ell = ell, cls = theory_cls, l_mode_index = 300)
cl_binned = bin_cls(cls  = one_draw_cls, num_tom = 5)
cl_cov = cls_calc_covariance(cls = cl_binned, ell = ell, l_mode_index = 300, A = 50)

In [28]:
cl_cov

array([[1.51179818e-21, 1.94951886e-25, 2.51397563e-29, 1.16859410e-25,
        1.50694468e-29, 9.03303222e-30, 7.61964687e-26, 9.82581238e-30,
        5.88985652e-30, 3.84039479e-30, 8.07921703e-26, 1.04184449e-29,
        6.24509639e-30, 4.07202375e-30, 4.31762314e-30],
       [1.94951886e-25, 3.68979672e-22, 9.51625441e-26, 9.39385562e-25,
        2.86426319e-26, 1.45224625e-28, 1.07425605e-24, 1.87355527e-26,
        1.30383569e-28, 1.08287130e-28, 1.11194228e-24, 1.98620703e-26,
        1.36152234e-28, 1.13452154e-28, 1.18846299e-28],
       [2.51397563e-29, 9.51625441e-26, 3.60222657e-22, 2.42272441e-28,
        9.17083748e-25, 2.33478540e-27, 2.77057199e-28, 1.04875591e-24,
        2.67000695e-27, 3.05335862e-27, 2.86776678e-28, 1.08554745e-24,
        2.76367380e-27, 3.16047388e-27, 3.27134686e-27],
       [1.16859410e-25, 9.39385562e-25, 2.42272441e-28, 2.55268446e-22,
        3.29904084e-26, 3.94636267e-26, 2.83725561e-24, 4.13219869e-28,
        1.30851550e-26, 2.86001977e-2

In [29]:
bin_combinations = ["bin_" for _ in range(5**2)]